In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
# train test split 
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv("preprocessed_data.csv")
df

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,hourly,employer_provided,min_salary,max_salary,avg_salary,job_state,age,job_title_tag,seniority,desc_len,python,R,spark,aws,excel,sql,django,hadoop,java,db,tableau,azure
0,0,Data Scientist,$74K - $139K,Intro (Use Font Arial 12):\r\nAs a Data Scient...,4.4,Applied Information Sciences,"Chevy Chase, MD",501 to 1000 Employees,1982,Unknown,IT Services,Information Technology,$50 to $100 million (USD),0,0,74,139,106.5,MD,39,data scientist,mid/other,755,0,0,0,0,0,1,0,0,0,1,0,1
1,1,Web Developer,$41K - $83K,We are the fastest growing employer of emergin...,4.0,Revature,"Tampa, FL",1001 to 5000 Employees,2003,Unknown,IT Services,Information Technology,$100 to $500 million (USD),0,0,41,83,62.0,FL,18,other,mid/other,625,0,0,0,0,0,0,0,0,0,0,0,0
2,2,Data Analyst,$43K - $88K,"HR Data Analyst - Direct Hire - Buffalo, NY\r\...",4.1,Systems Personnel,"Niagara Falls, NY",1 to 50 Employees,1996,Unknown,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),0,0,43,88,65.5,NY,25,analyst,mid/other,1366,0,0,0,0,0,1,0,0,0,0,1,0
3,3,Database Administrator,$68K - $136K,"Secure our Nation, Ignite your Future\r\nBecom...",4.2,ManTech International Corporation,"Fort Meade, MD",5001 to 10000 Employees,1968,Unknown,Research & Development,Business Services,$1 to $2 billion (USD),0,0,68,136,102.0,MD,53,other,mid/other,765,0,0,0,0,0,0,0,0,0,1,0,0
4,4,iOS Developer,$72K - $104K,Are you an iOS developer who loves to create m...,4.1,Garmin,"Olathe, KS",10000+ Employees,1989,Unknown,Consumer Products Manufacturing,Manufacturing,$2 to $5 billion (USD),0,0,72,104,88.0,KS,32,other,mid/other,929,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4912,4912,Front End Developer,$58K - $117K,US Citizenship is required\r\nFront end Develo...,-1.0,EDR Technology,"Ashburn, VA",Unknown,-1,Unknown,-1,-1,Unknown / Non-Applicable,0,0,58,117,87.5,VA,-1,other,mid/other,1314,0,0,0,0,0,0,0,0,1,1,0,0
4913,4913,Sr Software Quality Engineer,$78K - $147K,PRA Health Sciences is seeking the best and br...,4.0,PRA Health Sciences,"San Diego, CA",10000+ Employees,1976,Unknown,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),0,0,78,147,112.5,CA,45,software developer/engineer,senior,980,0,0,0,0,0,0,0,0,0,0,0,0
4914,4914,Lead Mobile Engineer,$79K - $162K,About Us\r\nWe are a fintech startup with a mi...,-1.0,Treecard,"Austin, TX",Unknown,-1,Unknown,-1,-1,Unknown / Non-Applicable,0,0,79,162,120.5,TX,-1,other,senior,812,0,0,0,0,0,0,0,0,0,0,0,0
4915,4915,Cloud Infrastructure Engineer (SRE/Java),$61K - $132K,"Every day, Global Payments makes it possible f...",3.5,Global Payments,"Columbus, GA",10000+ Employees,1967,Unknown,Financial Transaction Processing,Finance,$5 to $10 billion (USD),0,0,61,132,96.5,GA,54,other,senior,641,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'hourly',
       'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'job_state', 'age', 'job_title_tag', 'seniority', 'desc_len', 'python',
       'R', 'spark', 'aws', 'excel', 'sql', 'django', 'hadoop', 'java', 'db',
       'tableau', 'azure'],
      dtype='object')

In [8]:
df['Job Title'].unique()

array(['Data Scientist', 'Web Developer', 'Data Analyst', ...,
       'Lead Mobile Engineer', 'Cloud Infrastructure Engineer (SRE/Java)',
       'Development Chapter Lead in Retail'], dtype=object)

In [9]:
df_model = df[['Job Title', 'Rating','Company Name', 'Company Name', 'Size',
               'Type of ownership', 'Industry', 'Sector', 'Revenue', 'hourly', 'employer_provided',
               'avg_salary', 'job_state', 'age',
               'job_title_tag', 'seniority', 'desc_len', 'python', 'R', 'spark', 'aws',
               'excel', 'sql', 'django', 'hadoop', 'java', 'db', 'tableau', 'azure']]
               

In [10]:
dummy_df = pd.get_dummies(df_model)

X = dummy_df.drop('avg_salary', axis =1)
y = dummy_df.avg_salary.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [11]:
df['Job Description'].map(lambda x: len(x)).mean()

1031.6672767947937

In [36]:
df['avg_salary'].mean()

86.35946715476916

## RNN - GRU


In [37]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU

In [47]:

max_words = 10000
max_len = 1000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(df['Job Description'])
sequences = tok.texts_to_sequences(df['Job Description'])
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [113]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = GRU(128)(layer)
    layer = Bidirectional(layer)
    # layer = Dropout(0.2)(layer)
    # layer = Dense(64,name='FC2')(layer)
    # layer = Activation('sigmoid')(layer)
    # layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('relu')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [114]:

model = RNN()

model.summary()
model.compile(loss='mse',optimizer=RMSprop(),metrics=['mae', 'mse', 'msle'])

NameError: ignored

In [115]:
model.fit(sequences_matrix,df['avg_salary'],epochs=150,
                    batch_size=256,
                    verbose=1 ,validation_split=0.1)

Epoch 1/150
18/18 [==============================] - 2s 117ms/step - loss: 60.5776 - mae: 4.0615 - mse: 60.5776 - msle: 0.0058 - val_loss: 1093.4685 - val_mae: 24.6548 - val_mse: 1093.4685 - val_msle: 0.2525
Epoch 2/150
18/18 [==============================] - 2s 112ms/step - loss: 59.4960 - mae: 4.0388 - mse: 59.4960 - msle: 0.0057 - val_loss: 1099.2037 - val_mae: 24.9053 - val_mse: 1099.2037 - val_msle: 0.2456
Epoch 3/150
18/18 [==============================] - 2s 110ms/step - loss: 58.9754 - mae: 4.0308 - mse: 58.9754 - msle: 0.0057 - val_loss: 1089.7416 - val_mae: 24.7566 - val_mse: 1089.7416 - val_msle: 0.2485
Epoch 4/150
18/18 [==============================] - 2s 109ms/step - loss: 54.2114 - mae: 3.6356 - mse: 54.2114 - msle: 0.0049 - val_loss: 1099.7091 - val_mae: 24.7008 - val_mse: 1099.7091 - val_msle: 0.2513
Epoch 5/150
18/18 [==============================] - 2s 111ms/step - loss: 55.3328 - mae: 3.8317 - mse: 55.3328 - msle: 0.0053 - val_loss: 1058.8799 - val_mae: 24.2449 

In [120]:

test_sequences = tok.texts_to_sequences(
    ["""We are looking for a full time iOS Software Engineer to join our team in Olathe, KS. In this role, you’ll be responsible for architecting and developing complex software utilized by the suite of Garmin mobile applications. Responsibilities include:
Develop software using Swift and Objective-C
Utilize tools in the software development process such as Git, Gerrit, Bitbucket, Jira, Confluence, Xcode, and Jenkins
Create and implement designs for software components within a larger project"""])
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [121]:
pred = model.predict(test_sequences_matrix)
pred

array([[107.400795]], dtype=float32)

In [122]:

test_sequences = tok.texts_to_sequences(["""As a Data Scientist Architect you will be applying new and innovative methods to address some of our most important intelligence problems \
What You'll Be Doing:\
Knowledge and experience in designing and implementing AI apps and agents that use the Microsoft Azure Cognitive Services, Azure Bot Service, Azure Cognitive Search, and data storage options \
Analyzing requirements for AI solutions Experience with NoSQL databases Experience with applying Data Science techniques to overhead imagery (computer vision, object detection \
Ability to work autonomously and collaboratively as part of a team to both teach and learn every day"""])
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
pred = model.predict(test_sequences_matrix)
pred

array([[67.131035]], dtype=float32)

In [119]:
sequences_matrix[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   